# **ANÁLISIS EXPLORATORIO DE DATOS (EDA)**

## CONFIGURACIÓN INICIAL

Librerías

In [1]:
from plotly.subplots import make_subplots
from collections import defaultdict
import plotly.graph_objects as go
from plotly.offline import plot
from datetime import datetime
import plotly.express as px
import pandas as pd
import numpy as np

Lectura de datos

In [2]:
df = pd.read_csv('/content/Casos_positivos_de_COVID-19_en_Colombia._20240910.csv')

<ipython-input-2-56230ce0af5d>:1: DtypeWarning:

Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.



## RESUMEN DE DATOS

In [3]:
def detectar_valores_similares(df, columna):
    valor = df[columna].dropna().unique()
    valor_lower = [m.lower() for m in valor]

    similares = defaultdict(list)
    for i, v in enumerate(valor):
        for j, otro_v in enumerate(valor):
            if i != j and v.lower() == otro_v.lower():
                similares[v.lower()].append(v)
                similares[v.lower()].append(otro_v)

    return {k: list(set(v)) for k, v in similares.items()}

In [4]:
def analizar_datos_covid(df):
    num_registros = df.shape[0]
    num_columnas = len(df.columns)
    departamentos = df['Nombre departamento'].nunique()
    municipios = df['Nombre municipio'].nunique()
    columnas = df.columns.tolist()
    tipos_datos = df.dtypes.to_dict()

    datos_faltantes = df.isnull().sum()
    proporcion_faltantes = (datos_faltantes / num_registros) * 100

    info_df = f"Cantidad de registros: {num_registros}\n"
    info_df += f"Cantidad de columnas: {num_columnas}\n"
    info_df += f"\nCantidad de departamentos: {departamentos}\n"
    info_df += f"Cantidad de municipios: {municipios}\n\n"
    info_df += "Columnas (tipo dato, datos faltantes):\n"
    for columna in columnas:
        if tipos_datos[columna] == 'object':
            info_df += f"  - {columna}: string | {proporcion_faltantes[columna]:.2f}%\n"
        else:
            info_df += f"  - {columna}: {tipos_datos[columna]} | {proporcion_faltantes[columna]:.2f}%\n"

    # Verificar inconsistencias en fechas
    if 'fecha reporte web' in df.columns and 'fecha de notificacion' in df.columns:
        df['fecha reporte web'] = pd.to_datetime(df['fecha reporte web'], errors='coerce')
        df['fecha de notificacion'] = pd.to_datetime(df['fecha de notificacion'], errors='coerce')
        inconsistencias_fechas = (df['fecha reporte web'] < df['fecha de notificacion']).sum()
        info_df += f"\nInconsistencias en fechas (reporte < notificación): {inconsistencias_fechas}\n"

    # Verificar fechas futuras
    fecha_actual = datetime.now()
    df['fecha reporte web'] = pd.to_datetime(df['fecha reporte web'], errors='coerce')
    if 'fecha reporte web' in df.columns:
        fechas_futuras = (df['fecha reporte web'] > fecha_actual).sum()
        info_df += f"Fechas de reporte en el futuro: {fechas_futuras}\n"

    # Verificar valores atípicos en la edad
    if 'Edad' in df.columns:
        q1 = df['Edad'].quantile(0.25)
        q3 = df['Edad'].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (1.5 * iqr)
        upper_bound = q3 + (1.5 * iqr)
        atipicos_edad = ((df['Edad'] < lower_bound) | (df['Edad'] > upper_bound)).sum()
        info_df += f"\nValores atípicos en Edad ]{q1}, {q3}[: {atipicos_edad}\n"

    # Verificar edades imposibles
    edades_imposibles = ((df['Edad'] < 0) | (df['Edad'] > 120)).sum()
    info_df += f"Edades imposibles ]0, 120[: {edades_imposibles}\n"

    registros_por_sexo = df['Sexo'].value_counts().to_dict()
    registros_por_tipo_contagio = df['Tipo de contagio'].value_counts().to_dict()

    info_df += "Registros por sexo:\n"

    for sexo, count in registros_por_sexo.items():
        info_df += f"  - {sexo}: {count}\n"

    info_df += "\nRegistros por tipo de contagio:\n"
    for tipo_contagio, count in registros_por_tipo_contagio.items():
        info_df += f"  - {tipo_contagio}: {count}\n"

    # Detectar municipios con nombres similares (diferencias en mayúsculas/minúsculas)
    municipios_similares = detectar_valores_similares(df, 'Nombre municipio')
    if municipios_similares:
        info_df += "\nMunicipios con nombres similares (diferencias en mayúsculas/minúsculas):\n"
        for nombre_lower, variantes in municipios_similares.items():
            info_df += f"  - {nombre_lower}: {', '.join(variantes)}\n"

    # Detectar departamentos con nombres similares (diferencias en mayúsculas/minúsculas)
    departamentos_similares = detectar_valores_similares(df, 'Nombre departamento')
    if departamentos_similares:
        info_df += "\nDepartamentos con nombres similares (diferencias en mayúsculas/minúsculas):\n"
        for nombre_lower, variantes in departamentos_similares.items():
            info_df += f"  - {nombre_lower}: {', '.join(variantes)}\n"

    # Verificar inconsistencias entre estado y fecha de muerte
    if 'Estado' in df.columns and 'Fecha de muerte' in df.columns:
        inconsistencias_muerte = ((df['Estado'] == 'Fallecido') & (df['Fecha de muerte'].isnull())).sum()
        info_df += f"\nFallecidos sin fecha de muerte: {inconsistencias_muerte}\n"

        inconsistencias_vivos = ((df['Estado'] != 'Fallecido') & (df['Fecha de muerte'].notnull())).sum()
        info_df += f"No fallecidos con fecha de muerte: {inconsistencias_vivos}\n"

    # Verificar inconsistencias en el tipo de recuperación
    if 'Tipo de recuperación' in df.columns and 'Estado' in df.columns:
        inconsistencias_recuperacion = ((df['Estado'] == 'Recuperado') & (df['Tipo de recuperación'].isnull())).sum()
        info_df += f"Recuperados sin tipo de recuperación: {inconsistencias_recuperacion}\n"

    # Verificar duplicados
    duplicados = df.duplicated().sum()
    info_df += f"\nRegistros duplicados: {duplicados}\n"

    # Verificar consistencia entre tipo de contagio y país de procedencia
    if 'Tipo de contagio' in df.columns and 'País de procedencia' in df.columns:
        inconsistencias_procedencia = ((df['Tipo de contagio'] == 'Importado') & (df['País de procedencia'].isnull())).sum()
        info_df += f"Casos importados sin país de procedencia: {inconsistencias_procedencia}\n"

    return info_df

In [5]:
# Guardar el resumen actualizado
with open('/content/drive/MyDrive/Maestría Ingeniería de Sistemas/Semestre I/Minería de Datos/Compromisos/Homework #2/Resumen de datos.txt', 'w') as f:
  f.write(analizar_datos_covid(df))

## VISUALIZACIONES

In [6]:
# Colores y configuración de estilo
colores_paleta = {
    'primary_100': '#6B5B95',
    'primary_200': '#4B4068',
    'primary_300': '#E1DDEB',
    'accent_100': '#FF7F11',
    'accent_200': '#BE5800',
    'text_100': '#333333',
    'text_200': '#424242',
    'bg_100': '#E6E6FA',
    'bg_200': '#D2D2F6',
    'bg_300': '#FFFFFF'
}

# Resto del código
def preparar_datos(df):
    fecha_columnas = ['fecha reporte web', 'Fecha de inicio de síntomas', 'Fecha de recuperación', 'Fecha de muerte']
    for col in fecha_columnas:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    df['fecha_fin'] = df['Fecha de recuperación'].fillna(df['Fecha de muerte'])
    df['Dias en UCI'] = (df['fecha_fin'] - df['Fecha de inicio de síntomas']).dt.days
    df = df[df['Dias en UCI'].between(0, 365)]

    return df

def crear_visualizaciones_covid(df):
    df = preparar_datos(df)
    visualizaciones = []

    # 1. Evolución temporal de casos
    fig_casos = px.line(df.groupby('fecha reporte web').size().reset_index(name='Casos'),
                        x='fecha reporte web', y='Casos')
    # fig_casos.update_traces(line_color=colores_paleta['primary_200'])
    visualizaciones.append([fig_casos, 'Evolución temporal de casos de COVID-19'])

    # 2. Distribución de casos por edad
    fig_edad = px.histogram(df, x='Edad', nbins=20)
    visualizaciones.append([fig_edad, 'Distribución de casos por edad'])

    # 3. Casos por departamento (top 10)
    casos_por_depto = df['Nombre departamento'].value_counts().nlargest(10)
    fig_depto = px.bar(x=casos_por_depto.index, y=casos_por_depto.values)
    visualizaciones.append([fig_depto, 'Top 10 departamentos con más casos'])

    # 4. Proporción de casos por tipo de contagio
    fig_tipo_contagio = px.pie(df, names='Tipo de contagio',
                               color_discrete_sequence=[colores_paleta['primary_100'], colores_paleta['accent_100']])
    visualizaciones.append([fig_tipo_contagio, 'Proporción de casos por tipo de contagio'])

    # 5. Evolución temporal de casos por estado
    df_estado = df.groupby(['fecha reporte web', 'Estado']).size().unstack(fill_value=0)
    fig_estado = px.area(df_estado)
    visualizaciones.append([fig_estado, 'Evolución temporal de casos por estado'])

    # 6. Boxplot de edad por estado
    fig_edad_estado = px.box(df, x='Estado', y='Edad')
    visualizaciones.append([fig_edad_estado, 'Distribución de edad por estado'])

    # 7. Casos por día de la semana
    df['dia_semana'] = df['fecha reporte web'].dt.dayofweek
    casos_por_dia_semana = df.groupby('dia_semana').size()
    fig_dia_semana = px.bar(x=casos_por_dia_semana.index, y=casos_por_dia_semana.values,
                             labels={'x': 'Día de la semana', 'y': 'Número de casos'})
    visualizaciones.append([fig_dia_semana, 'Casos por día de la semana'])

    # 8. Scatter plot de Edad vs Días en UCI
    fig_scatter = px.scatter(df, x='Edad', y='Dias en UCI', color='Estado')
    visualizaciones.append([fig_scatter, 'Relación entre Edad y Días en UCI (calculado)'])

    # 9. Casos acumulados vs. Nuevos casos diarios
    df_daily = df.groupby('fecha reporte web').size().reset_index(name='Nuevos_casos')
    df_daily['Casos_acumulados'] = df_daily['Nuevos_casos'].cumsum()
    fig_acumulados = px.scatter(df_daily, x='Casos_acumulados', y='Nuevos_casos',
                                labels={'Casos_acumulados': 'Casos acumulados', 'Nuevos_casos': 'Nuevos casos diarios'})
    visualizaciones.append([fig_acumulados, 'Casos acumulados vs. Nuevos casos diarios'])

    # 10. Pirámide poblacional de casos
    df_piramide = df.groupby(['Sexo', pd.cut(df['Edad'], bins=range(0, 101, 10))]).size().unstack(level=0)
    fig_piramide = go.Figure()
    for sexo in df_piramide.columns:
        fig_piramide.add_trace(go.Bar(
            y=df_piramide.index.astype(str),
            x=df_piramide[sexo] if sexo == 'M' else -df_piramide[sexo],
            name=sexo,
            orientation='h'
        ))
    fig_piramide.update_layout(barmode='relative',
                               bargap=0.1,
                               bargroupgap=0)
    visualizaciones.append([fig_piramide, 'Pirámide poblacional de casos de COVID-19'])

    return visualizaciones

def guardar_visualizaciones_individual(visualizaciones, carpeta='graficas'):
    import os
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    for i, fig in enumerate(visualizaciones, 1):
        nombre_archivo = os.path.join(carpeta, fig[1])
        plot(fig[0], filename=nombre_archivo, auto_open=False)
        print(f"Gráfico {i} guardado en {nombre_archivo}")

# Uso de las funciones
visualizaciones = crear_visualizaciones_covid(df)
guardar_visualizaciones_individual(visualizaciones)


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

<ipython-input-6-7bc8e5754c75>:62: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-7bc8e5754c75>:80: 

Gráfico 1 guardado en graficas/Evolución temporal de casos de COVID-19
Gráfico 2 guardado en graficas/Distribución de casos por edad
Gráfico 3 guardado en graficas/Top 10 departamentos con más casos


/usr/local/lib/python3.10/dist-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `graficas/Top 10 departamentos con más casos` didn't end with .html. Adding .html to the end of your file.

/usr/local/lib/python3.10/dist-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `graficas/Proporción de casos por tipo de contagio` didn't end with .html. Adding .html to the end of your file.



Gráfico 4 guardado en graficas/Proporción de casos por tipo de contagio
Gráfico 5 guardado en graficas/Evolución temporal de casos por estado


/usr/local/lib/python3.10/dist-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `graficas/Evolución temporal de casos por estado` didn't end with .html. Adding .html to the end of your file.

/usr/local/lib/python3.10/dist-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `graficas/Distribución de edad por estado` didn't end with .html. Adding .html to the end of your file.



Gráfico 6 guardado en graficas/Distribución de edad por estado
Gráfico 7 guardado en graficas/Casos por día de la semana


/usr/local/lib/python3.10/dist-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `graficas/Casos por día de la semana` didn't end with .html. Adding .html to the end of your file.

/usr/local/lib/python3.10/dist-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `graficas/Relación entre Edad y Días en UCI (calculado)` didn't end with .html. Adding .html to the end of your file.



Gráfico 8 guardado en graficas/Relación entre Edad y Días en UCI (calculado)
Gráfico 9 guardado en graficas/Casos acumulados vs. Nuevos casos diarios
Gráfico 10 guardado en graficas/Pirámide poblacional de casos de COVID-19


/usr/local/lib/python3.10/dist-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `graficas/Casos acumulados vs. Nuevos casos diarios` didn't end with .html. Adding .html to the end of your file.

/usr/local/lib/python3.10/dist-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `graficas/Pirámide poblacional de casos de COVID-19` didn't end with .html. Adding .html to the end of your file.



## LIMPIEZA DE DATOS

Selección de columnas relevantes

In [7]:
df_limpieza = df[['fecha reporte web', 'ID de caso', 'Fecha de notificación',
       'Código DIVIPOLA departamento', 'Nombre departamento',
       'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad',
       'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
       'Ubicación del caso', 'Estado', 'Recuperado', 'Fecha de inicio de síntomas',
       'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación',
       'Tipo de recuperación']].copy()

Renombramiento de columnas

In [8]:
df_limpieza.rename(columns={
    'ID de caso': 'id',
    'fecha reporte web': 'fecha_reporte_web',
    'Fecha de notificación': 'fecha_notificacion',
    'Código DIVIPOLA departamento': 'cod_departamento',
    'Nombre departamento': 'departamento',
    'Código DIVIPOLA municipio': 'cod_municipio',
    'Nombre municipio': 'municipio',
    'Edad': 'edad',
    'Unidad de medida de edad': 'unidad_medida_edad',
    'Sexo': 'sexo',
    'Tipo de contagio': 'tipo_contagio',
    'Ubicación del caso': 'ubicacion_caso',
    'Estado': 'estado',
    'Recuperado': 'recuperado',
    'Fecha de inicio de síntomas': 'fecha_inicio_sintomas',
    'Fecha de muerte': 'fecha_muerte',
    'Fecha de diagnóstico': 'fecha_diagnostico',
    'Fecha de recuperación': 'fecha_recuperacion',
    'Tipo de recuperación': 'tipo_recuperacion'
}, inplace=True)

Reordenamiento de columnas

In [9]:
new_column_order = [
    'id', 'fecha_reporte_web', 'fecha_notificacion',
    'fecha_inicio_sintomas', 'fecha_muerte',
    'fecha_diagnostico', 'fecha_recuperacion',
    'cod_departamento', 'departamento', 'cod_municipio',
    'municipio', 'ubicacion_caso', 'edad',
    'unidad_medida_edad', 'sexo', 'tipo_contagio',
    'estado', 'recuperado', 'tipo_recuperacion',
]
df_limpieza = df_limpieza[new_column_order]

Tipo de datos

In [10]:
df_limpieza = df_limpieza.astype({
    'id': 'string',
    'fecha_reporte_web': 'datetime64[ns]',
    'fecha_notificacion': 'datetime64[ns]',
    'fecha_inicio_sintomas': 'datetime64[ns]',
    'fecha_muerte': 'datetime64[ns]',
    'fecha_diagnostico': 'datetime64[ns]',
    'fecha_recuperacion': 'datetime64[ns]',
    'cod_departamento': 'int64',
    'departamento': 'string',
    'cod_municipio': 'int64',
    'municipio': 'string',
    'ubicacion_caso': 'string',
    'edad': 'int64',
    'unidad_medida_edad': 'string',
    'sexo': 'string',
    'tipo_contagio': 'string',
    'estado': 'string',
    'recuperado': 'string',
    'tipo_recuperacion': 'string'
})

Normalizar de valores tipo string

In [11]:
import unicodedata

def normalize_text(text, type='upper'):
  """Normaliza el texto eliminando tildes, caracteres especiales y capitalizando."""
  if isinstance(text, str):
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    if type == 'upper':
      text = text.upper()
    elif type == 'lower':
      text = text.lower()
    elif type == 'title':
      text = text.title()
    return text
  return text

df_limpieza['departamento'] = df_limpieza['departamento'].apply(lambda x: normalize_text(x))
df_limpieza['municipio'] = df_limpieza['municipio'].apply(lambda x: normalize_text(x))
df_limpieza['sexo'] = df_limpieza['sexo'].apply(lambda x: normalize_text(x))
df_limpieza['ubicacion_caso'] = df_limpieza['ubicacion_caso'].apply(lambda x: normalize_text(x, 'title'))
df_limpieza['estado'] = df_limpieza['estado'].apply(lambda x: normalize_text(x, 'title'))
df_limpieza['recuperado'] = df_limpieza['recuperado'].apply(lambda x: normalize_text(x, 'title'))
df_limpieza['tipo_recuperacion'] = df_limpieza['tipo_recuperacion'].apply(lambda x: normalize_text(x))

## IMPUTACIÓN DE DATOS

Fechas nulas posibles de imputar

In [12]:
df_limpieza['fecha_diagnostico'].fillna(df_limpieza['fecha_reporte_web'], inplace=True)

Imputación por moda

In [13]:
categorical_columns = ['ubicacion_caso', 'tipo_recuperacion']
for column in categorical_columns:
    mode_value = df_limpieza[column].mode()[0]
    df_limpieza[column].fillna(mode_value, inplace=True)

Casos especiales

In [14]:
df_limpieza['recuperado'] = np.where(
    df_limpieza['fecha_recuperacion'].notnull(), 'Recuperado',
    np.where(df_limpieza['fecha_muerte'].notnull(), 'Fallecido', 'Activo')
)

moda_estado = df_limpieza['estado'].mode()[0]
df_limpieza['estado'] = np.where(
    df_limpieza['fecha_muerte'].notnull(), 'Fallecido',
    np.where(
        df_limpieza['estado'].isnull(), moda_estado,
        df_limpieza['estado']
    )
)

## INFORME

Se realizó un análisis descriptivo inicial del conjunto de datos para obtener una visión general de su contenido. Esto incluyó:


*   Cantidad de registros y columnas.
*   Tipos de datos de cada columna.
*   Cantidad de departamentos y municipios.
*   Identificación de valores faltantes.
*   Detección de inconsistencias en fechas (fechas de reporte anteriores a fechas de notificación, fechas futuras).
*   Detección de valores atípicos en la edad (edades fuera del rango intercuartil, edades imposibles).
*   Análisis de la distribución de casos por sexo y tipo de contagio.
*   Detección de posibles duplicados.
*   Identificación de inconsistencias entre estado del paciente y fecha de muerte o tipo de recuperación.


Posteriormente, se crearon diversos gráficos para comprender mejor los patrones y tendencias en los datos. Las visualizaciones generadas fueron:


*   **Evolución temporal de casos**: cantidad de casos de COVID-19 a lo largo del tiempo. Se utiliza un gráfico de líneas para representar la tendencia de los casos a lo largo de las fechas de reporte. Permite identificar periodos con picos de contagios y la evolución general de la pandemia.
*   **Distribución de casos por edad**: histograma que muestra la distribución de los casos de COVID-19 según la edad de los pacientes. Permite identificar los grupos de edad más afectados por la enfermedad y analizar la distribución de la edad en la población contagiada.
*   **Casos por departamento (top 10)**: gráfico de barras que muestra los 10 departamentos con mayor número de casos de COVID-19. Permite identificar las zonas geográficas con mayor incidencia de la enfermedad y focalizar esfuerzos en áreas específicas.
*   **Proporción de casos por tipo de contagio**: un gráfico circular que muestra la proporción de casos de COVID-19 según el tipo de contagio (importado, relacionado, en estudio, etc.). Permite entender cómo se propaga la enfermedad en la población y la importancia relativa de cada tipo de contagio.
*   **Evolución temporal de casos por estado**: un gráfico de área que muestra la evolución temporal de los casos de COVID-19 según el estado del paciente (recuperado, fallecido, activo). Permite observar la progresión de la enfermedad en el tiempo y la proporción de pacientes en cada estado.
*   **Gráfico de caja de edad por estado**: un diagrama de caja que muestra la distribución de la edad de los pacientes según su estado (recuperado, fallecido, activo). Permite comparar la distribución de la edad entre los diferentes estados e identificar posibles diferencias significativas.
*   **Casos por día de la semana**: gráfico de barras para mostrar la distribución de casos de COVID-19 según el día de la semana. Este gráfico permite identificar si existe algún patrón o tendencia en la cantidad de casos reportados en diferentes días de la semana.
*   **Gráfico de dispersión de Edad vs días enfermo (calculado)**: gráfico de dispersión que muestra la relación entre la edad de los pacientes y los días que pasaron bajo observación antes de recuperación o muerte. Permite identificar si existe una correlación entre la edad y la gravedad de la enfermedad, representada por días de incapacidad.
*   **Casos acumulados vs. Nuevos casos diarios**: gráfico de dispersión que muestra la relación entre los casos acumulados y los nuevos casos diarios de COVID-19. Permite analizar la tendencia de la pandemia y si se está produciendo un aumento o disminución en la velocidad de propagación.
*   **Pirámide poblacional de casos de COVID-19**: gráfico de pirámide poblacional que muestra la distribución de casos de COVID-19 por sexo y grupos de edad. Permite analizar la distribución de la enfermedad en la población y comparar la afectación entre hombres y mujeres en diferentes grupos de edad.


Se realizó un proceso de limpieza de datos para preparar el conjunto de datos para su posterior análisis. Este proceso incluyó:


*   Selección de columnas relevantes: se seleccionaron las columnas que se consideraron más relevantes para el análisis y que cuentan con un gran porcentaje de valores no nulos.
*   Renombramiento de columnas: se renombraron las columnas para que tuvieran nombres más concisos y fáciles de usar en el análisis.
*   Reordenamiento de columnas: se reordenaron las columnas para que tuvieran un orden más lógico.
*   Conversión de tipos de datos: se aseguraron de que todas las columnas tuvieran el tipo de datos correcto.
*   Normalización de valores de texto: se normalizaron los valores de texto en las columnas de tipo string para eliminar tildes, caracteres especiales y asegurar la consistencia en la capitalización. Debido a que los datos presentaban errores de ingreso de información por caracteres diferentes.


Finalmente, se utilizaron diferentes métodos de imputación para manejar los valores faltantes en el conjunto de datos.



*   **Imputación por fecha de reporte**: se imputaron las fechas de diagnóstico faltantes utilizando la fecha de reporte web, asumiendo que el la fecha de reporte coicidia con la fecha en que el paciente fue diagnisticado con COVID-19.
*   **Imputación por moda**: para las variables categóricas `ubicacion_caso` y `tipo_recuperacion`, se utilizó la moda para imputar los valores faltantes. La imputación por moda es una técnica simple y comúnmente utilizada para variables categóricas, especialmente cuando la moda representa una proporción significativa de los datos.
*   **Imputación deductiva**: se imputaron los valores de las columnas recuperado y estado utilizando la información disponible en otras columnas, como las fechas de muerte y recuperación. En base al conocimiento del campo y las relaciones lógicas entre las variables, se planteo una imputación consistente con la realidad.